In [2]:
!pip install tensorflow-model-optimization
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 6.3 MB/s eta 0:00:00


In [4]:
# Load Oxford Flowers 102 dataset
dataset, info = tfds.load("oxford_flowers102", as_supervised=True, with_info=True)
num_classes = info.features['label'].num_classes

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0  # Normalize
    return image, label

# Prepare datasets
train_data = dataset['train'].map(preprocess).batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_data = dataset['validation'].map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/6149 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.GIRZPB_2.1.1/oxford_flowers102-test.tfrecord*…

Generating validation examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.GIRZPB_2.1.1/oxford_flowers102-validation.tfr…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [5]:
# Define a CNN model using MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the baseline model
model.fit(train_data, validation_data=val_data, epochs=5)

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/5
32/32 [==============================] - 14s 177ms/step - loss: 4.3267 - accuracy: 0.1157 - val_loss: 3.1149 - val_accuracy: 0.3608
Epoch 2/5
32/32 [==============================] - 6s 123ms/step - loss: 2.1439 - accuracy: 0.6471 - val_loss: 2.0796 - val_accuracy: 0.6196
Epoch 3/5
32/32 [==============================] - 5s 111ms/step - loss: 1.1880 - accuracy: 0.8598 - val_loss: 1.6128 - val_accuracy: 0.6922
Epoch 4/5
32/32 [==============================] - 8s 206ms/step - loss: 0.7393 - accuracy: 0.9441 - val_loss: 1.3426 - val_accuracy: 0.7373
Epoch 5/5
32/32 [==============================] - 5s 122ms/step - loss: 0.4902 - accuracy: 0.9804 - val_loss: 1.1994 - val_accuracy: 0.7539


In [6]:
# Apply "Gradient-Based" Pruning (Using Polynomial Decay)
def apply_gradient_based_pruning(model):
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.2,   # Start with 20% sparsity
            final_sparsity=0.6,     # End with 60% sparsity
            begin_step=0,
            end_step=2000           # Gradual pruning over training steps
        )
    }

    pruned_model = prune_low_magnitude(model, **pruning_params)
    return pruned_model

# Apply pruning and recompile
pruned_model = apply_gradient_based_pruning(model)
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fine-tune pruned model
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]

pruned_model.fit(train_data, validation_data=val_data, epochs=5, callbacks=callbacks)

Epoch 1/5
32/32 [==============================] - 49s 220ms/step - loss: 0.4129 - accuracy: 0.9745 - val_loss: 1.0515 - val_accuracy: 0.7598
Epoch 2/5
32/32 [==============================] - 7s 166ms/step - loss: 0.2001 - accuracy: 0.9971 - val_loss: 0.9387 - val_accuracy: 0.7725
Epoch 3/5
32/32 [==============================] - 6s 157ms/step - loss: 0.1243 - accuracy: 1.0000 - val_loss: 0.8761 - val_accuracy: 0.7853
Epoch 4/5
32/32 [==============================] - 10s 258ms/step - loss: 0.8741 - accuracy: 0.8255 - val_loss: 1.3628 - val_accuracy: 0.6784
Epoch 5/5
32/32 [==============================] - 7s 154ms/step - loss: 0.5440 - accuracy: 0.9304 - val_loss: 1.2029 - val_accuracy: 0.7049


In [7]:
# Evaluate pruned model
_, pruned_acc = pruned_model.evaluate(val_data)
print(f"Pruned Model Accuracy: {pruned_acc:.4f}")

32/32 [==============================] - 2s 59ms/step - loss: 1.2029 - accuracy: 0.7049
Pruned Model Accuracy: 0.7049
